# Code

### Config - imports, constants, and mappings

In [203]:
import astropy
from astropy.io import fits
from astropy.table import Table

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

output_dir = "../output/"

""" 
The groupings below map specific data-based claimed types to larger 
groupings. For example: nIa, Ia, Ia*, and Ia-HV all map to Ia.
Any new claimed types to be considered in the analysis need to be added 
here and mapped to a larger group, which also needs to be in cat_code
"""
groupings = {
#     Ia
    'nIa' : 'Ia',
    'Ia' : 'Ia',
    'Ia*' :'Ia',
    'Ia-HV' :'Ia',
#     'Ia-91T'
    'Ia-91T' : 'Ia-91T',
#     'Ia-91bg'
    'Ia-91bg' : 'Ia-91bg',
#     'Ib/Ic'
    'IIb/Ib/Ic (Ca rich)' : 'Ib/Ic',
    'Ib/Ic (Ca rich)' : 'Ib/Ic',
    'Ib (Ca rich)' : 'Ib/Ic',
#     'Ia CSM'
    'Ia CSM' : 'Ia CSM',
#     'Ia-02cx'
    'Ia-02cx' : 'Ia-02cx',
    'Iax[02cx-like]' : 'Ia-02cx',
#     'II P'
    'II P' : 'II P',
    'II-p' : 'II P',
    'II Pec' : 'II P',
    'II P Pec' : 'II P',
    'II?' : 'II P',
    'II Pec?' : 'II P',
    'IIP?' : 'II P',
    'II' : 'II P',
#     'IIn'
    'IIn' : 'IIn',
    'IIn Pec' : 'IIn',
    'LBV to IIn' : 'IIn',
    'IIn-pec/LBV' : 'IIn',
    'IIn?' : 'IIn',
#     'Ib'
    'Ib' : 'Ib',
    'Ibn' : 'Ib',
#     'Ic'
    'Ic' : 'Ic',
    'Ic?' : 'Ic',
    'Ic-lum?' : 'Ic',
    'Ic Pec' : 'Ic',
#     Ic BL
    'Ic BL' : 'Ic BL',
    'BL-Ic' : 'Ic BL',
#     'SLSN-II?'
    'SLSN-II?' : 'SLSN-II?',
#     'SLSN-I'
    'SLSN-I' : 'SLSN-I',
#     'LGRB'
    'LGRB' : 'LGRB'
}



"""
Claimed Type categories, mapped to integer values.
The integer values are used by the ML model.
""" 
cat_code = {
    'LGRB': 1,
    'Ia' : 2, 
    'II P': 3, 
    'Ib' : 4,
    'Ic' : 5, 
    'IIn':6, 
    'SLSN-I' : 7, 
    'Ia-02cx' : 8,
    'Ic BL' : 9, 
    'SLSN-II?': 10, 
    'Ia-91bg' : 11, 
    'Ia-91T' : 12, 
    'Ia CSM' : 13
}

### Prep Data

##### Convert .fits data to Pandas DataFrame

In [212]:
def collect_data(file = '../data/osc+otc-Assembled.fits'):
    dat = Table.read(file, format='fits')
    df_bytes = dat.to_pandas()  # Convert to pandas dataframe
    df = pd.DataFrame()     # Init empty dataframe for converted types

    # Convert byte columns to strings
    for column in df_bytes:
        if df_bytes[column].dtype == np.dtype('object'):
            df[column + "_str"] = df_bytes[column].str.decode("utf-8")
            df[column] = df[column + "_str"].copy()
            df.drop(column + "_str", axis = 1, inplace = True)
        else:
            df[column] = df_bytes[column]

    # Prints sum of NULL values by column
    df.isnull().sum().to_csv(output_dir + "Missing_Values.csv")
    return df

##### Group by Claimed Type and Redshift

In [225]:
def group_cts(valid_df):
    # Group claimed types into supergroups, defined in groupings dict
    valid_df['claimedtype_group'] = valid_df.apply(lambda row:  
                                             groupings[row.claimedtype] 
                                             if row.claimedtype in groupings 
                                             else None,
                                             axis=1)

    # Dataframe of claimed types that do not have group
    ungrouped_types = list(set(valid_df.claimedtype.unique()) - set(groupings.keys()))
    print(str(len(ungrouped_types)) + " rows with ungrouped claimed type.")

    # Claimed Types that need to be grouped
    add_cts = []
    for ct in list(valid_df.claimedtype.unique()):
        if "," not in ct and ct != '' and ct not in groupings:
            add_cts.append(ct)
    # Drop rows with no grouped claimed type      
    valid_df = valid_df[~valid_df['claimedtype_group'].isnull()]
    print ("Remaining rows in valid dataframe: " + str(valid_df.shape[0]))
    
    return valid_df


# Machine Learning Analysis

In [414]:
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

import sklearn.metrics as skm

from sklearn.ensemble import RandomForestClassifier


#### Set up source and target data

In [182]:
# Set up Target dataframe
def set_up_target(df_rs_band):
    df_analyze = df_rs_band.copy()

    # Split out claimedtype_group into new dataframe
    y = df_analyze.claimedtype_group.to_frame(name='group')
    # Use category code numbers from cat_code dict, instead of strings
    y['cat_code'] = y.apply(lambda row:  cat_code[row.group] , axis=1)
    y = y.drop('group', axis = 1)
    
    return y


def set_up_source(df_analyze):
    X = df_analyze.copy()
    # Drop target column, and columns related to redshift or 
    # specific galactic identity (like ra and dec)
    
#     cols_to_drop = [
#     #             Target
#                 'claimedtype_group', 
#                 'claimedtype',

#     #             Galactic info
#                 'dec',
#                 'ra',
#                 'hostra',
#                 'hostdec',
#                 'hostoffsetang',
#                 'hostoffsetdist',
#                 'maxdate',
#                 'hostoffsetdist',
#                 'name', 
#                 'host',
#                 'alias',
#                 'NED_identifier', 
#                 'NED_flags',
#                 'NED_searched',
#                 'discoverdate',
#                 'hostoffsetang',
#                 'hostdec',
        
#                 'HyperLEDA_objname',
#                 'HyperLEDA_f_astrom',
#                 'HyperLEDA_pgc',
        
#                 'ebv',
#                 'velocity',
#                 'SDSS_MJD',
#                 'SDSS_Plate',
#                 'SDSS_FiberID',
        

#     #             Redshift info
#                 'redshift',    
#                 'lumdist', 
#                 'maxabsmag', 
#                 'maxappmag'
#                 ]

    X = X.drop(['redshift', 'claimedtype_group'], axis = 1)
    
    
    # Encode all letters as numbers
    le = LabelEncoder()
    encoded_series = X[X.columns[:]].apply(le.fit_transform)
    X = encoded_series
    
    return X

def split_train_test(X, y):
    # Split Training and Testing Data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=2)
    return X_train, X_test, y_train, y_test

#### Random Forest Classifier

In [158]:
def run_rm(X_train, X_test, y_train, y_test):
    clf = RandomForestClassifier(max_depth = 8, random_state = 0, class_weight = "balanced")
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    return clf, predictions

def get_performance(y_test, predictions):
    return classification_report(y_test, predictions)

def get_feature_importance(clf, X_train):
    feature_importances = pd.DataFrame(clf.feature_importances_,
                                       index = X_train.columns,
                                       columns=['importance'])
    feature_importances = feature_importances.sort_values('importance', 
                                                          ascending = False)
    return feature_importances

### Run ML Model

In [395]:
def run_analysis(data_df, col_list, min_redshift, max_redshift):
#     Collect and prep data
    grouped_df = group_cts(data_df)
    valid_df = grouped_df.copy()
    
#     Filter on specific list of columns to consider
    col_list = col_list + ['redshift', 'claimedtype_group']
    valid_df = valid_df[col_list]
    
#     Filter on specific redshift bin
    valid_df = valid_df[(valid_df.redshift != 'nan') &
                    (valid_df.redshift.str.contains(',') == False)]

    valid_df['redshift'] = valid_df['redshift'].apply(pd.to_numeric)

    df_rs_band = valid_df.loc[(valid_df.redshift > min_redshift) 
                        & (valid_df.redshift <= max_redshift)]
    print(str(df_rs_band.shape[0]) + " rows in redshift band: " + str(min_redshift) + " - " + str(max_redshift))

#     Run through ML
    y = set_up_target(df_rs_band)
    X = set_up_source(df_rs_band)
    X_train, X_test, y_train, y_test = split_train_test(X, y)

    clf, predictions = run_rm(X_train, X_test, y_train, y_test)
    get_feature_importance(clf, X_train)
    print(classification_report(y_test, predictions))
    return clf, X_train, y_test, predictions

# Run Analysis

Run the entire analysis using the cells below. Uses a sample of the original data with redshift within min_redshift and max_redshift, and using only fields in the col_list of the analysis. 

col_list - the list of data fields to use in the machine learning algorithm. They have been subdivided into spectral columns (available in the cell below). The reason for this is to consider only the spectra in the analysis. Considering any other fields leads to a higher risk of having a biased algorithm favoring redshift-driven characteristics. 

min_redshift - The minimum redshift to consider for the data sample, exclusive

max_redshift - The maximum redshift to consider for the data sample, inclusive

In [417]:
data_df = collect_data()

In [ ]:
clf, X_train, y_test, predictions = run_analysis(data_df = data_df,
                            col_list = ned_sdss_cols,  
                            min_redshift = 0.2, 
                            max_redshift = 0.3)
get_feature_importance(clf, X_train)

In [442]:
def run_over_all_redshifts(sel_cols, valid_data):
    for col_name in sel_cols:
        valid_data = valid_data[valid_data[col_name].notnull()]

    rs_mins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
    rows = []
    for rs_min in rs_mins:
        rs_max = rs_min + 0.1
        clf, X_train, y_test, predictions = run_analysis(data_df = data_df,
                                col_list = sel_cols,  
                                min_redshift = rs_min, 
                                max_redshift = rs_max)

        df = get_feature_importance(clf, X_train)
        col_vals = []
        for col_name in sel_cols:
            col_vals.append(df[df.index == col_name].importance[0])

        types = list(range(1,14)) # list types to return, 1 to 13
        recalls = list(recall_score(y_test, predictions, labels = types, average = None)) 
        ps = list(precision_score(y_test, predictions, labels = types, average = None)) 
        f1 = list(f1_score(y_test, predictions, labels = types, average = None))
        class_2_recall = recalls[1]
        class_3_recall = recalls[2]
        class_2_ps = ps[1] 
        class_3_ps = ps[2]    
        class_2_f1 = f1[1]
        class_3_f1 = f1[2]
        row = [rs_min, rs_max] + col_vals + [class_2_recall, class_3_recall, class_2_ps, class_3_ps, class_2_f1, class_3_f1]
        rows.append(row)
        
    return rows


In [444]:
ned_mass_cols = ['NED_2MASS_J', 'NED_2MASS_H', 'NED_2MASS_Ks']

ned_sdss_cols = ['NED_SDSS_u', 'NED_SDSS_g', 'NED_SDSS_r', 'NED_SDSS_i', 'NED_SDSS_z']

ned_galex_cols = ['NED_GALEX_NUV', 'NED_GALEX_FUV']

ned_iras_cols = ['NED_IRAS_12m', 'NED_IRAS_25m', 'NED_IRAS_60m', 'NED_IRAS_100m', 'NED_HI_21cm', 'NED_1p4GHz']

allwise_cols = ['AllWISE_W1', 'AllWISE_W2', 'AllWISE_W3', 'AllWISE_W4']

firefly_cols = ['Firefly_Chabrier_MILES_Age_LW',
 'Firefly_Chabrier_MILES_Metallicity_LW',
 'Firefly_Chabrier_MILES_logMstar',
 'Firefly_Chabrier_MILES_nSSP',
 'Firefly_Chabrier_MILES_spm_EBV']

mpa_cols = [col for col in list(data_df) if "MPAJHU" in col] # MPAJHU
# HyperLEDA

# GalaxyZoo
zoo_cols = [col for col in list(data_df) if "Zoo" in col] # MPAJHU


In [456]:
mpa_cols

['MPAJHU_ARIII7135_Eqw',
 'MPAJHU_ARIII7135_Eqw_Err',
 'MPAJHU_HEI_5876_Eqw',
 'MPAJHU_HEI_5876_Eqw_Err',
 'MPAJHU_H_ALPHA_Eqw',
 'MPAJHU_H_ALPHA_Eqw_Err',
 'MPAJHU_H_BETA_Eqw',
 'MPAJHU_H_BETA_Eqw_Err',
 'MPAJHU_H_DELTA_Eqw',
 'MPAJHU_H_DELTA_Eqw_Err',
 'MPAJHU_H_GAMMA_Eqw',
 'MPAJHU_H_GAMMA_Eqw_Err',
 'MPAJHU_NEIII_3869_Eqw',
 'MPAJHU_NEIII_3869_Eqw_Err',
 'MPAJHU_NII_6548_Eqw',
 'MPAJHU_NII_6548_Eqw_Err',
 'MPAJHU_NII_6584_Eqw',
 'MPAJHU_NII_6584_Eqw_Err',
 'MPAJHU_OIII_4363_Eqw',
 'MPAJHU_OIII_4363_Eqw_Err',
 'MPAJHU_OIII_4959_Eqw',
 'MPAJHU_OIII_4959_Eqw_Err',
 'MPAJHU_OIII_5007_Eqw',
 'MPAJHU_OIII_5007_Eqw_Err',
 'MPAJHU_OII_3726_Eqw',
 'MPAJHU_OII_3726_Eqw_Err',
 'MPAJHU_OII_3729_Eqw',
 'MPAJHU_OII_3729_Eqw_Err',
 'MPAJHU_OI_6300_Eqw',
 'MPAJHU_OI_6300_Eqw_Err',
 'MPAJHU_SII_6717_Eqw',
 'MPAJHU_SII_6717_Eqw_Err',
 'MPAJHU_ARIII7135_Flux',
 'MPAJHU_ARIII7135_Flux_Err',
 'MPAJHU_HEI_5876_Flux',
 'MPAJHU_HEI_5876_Flux_Err',
 'MPAJHU_H_ALPHA_Flux',
 'MPAJHU_H_ALPHA_Flux_Err',
 'MPAJ

In [454]:
cur_cols = zoo_cols
rows = run_over_all_redshifts(cur_cols, data_df.copy())
report = pd.DataFrame(rows, columns = ['min redshift', 'max redshift'] + cur_cols + ['Ia Recall',  'II P Recall', 'Ia Precision', 'II P Precision', 'Ia F1', 'II P F1'])
report.round(decimals = 2).to_csv("../output/performance/zoo_cols.csv")


0
391 rows with ungrouped claimed type.
Remaining rows in valid dataframe: 15959
4230 rows in redshift band: 0 - 0.1
             precision    recall  f1-score   support

          1       0.00      0.00      0.00         0
          2       0.74      0.43      0.54       911
          3       0.33      0.39      0.36       351
          4       0.00      0.00      0.00        33
          5       0.07      0.22      0.11        55
          6       0.04      0.22      0.07        37
          9       0.00      0.00      0.00         4
         11       0.00      0.00      0.00         2
         12       0.00      0.00      0.00         3

avg / total       0.57      0.39      0.45      1396

0.1


/Users/marina/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
/Users/marina/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/marina/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/marina/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/marina/anaconda3

391 rows with ungrouped claimed type.
Remaining rows in valid dataframe: 15959
1272 rows in redshift band: 0.1 - 0.2
             precision    recall  f1-score   support

          1       0.00      0.00      0.00         1
          2       0.91      0.80      0.85       335
          3       0.47      0.75      0.58        71
          4       0.00      0.00      0.00         1
          5       0.00      0.00      0.00         1
          6       0.00      0.00      0.00         8
          7       0.00      0.00      0.00         1
          8       0.00      0.00      0.00         1
         12       0.00      0.00      0.00         1

avg / total       0.81      0.77      0.78       420

0.2


/Users/marina/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
/Users/marina/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/marina/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/marina/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/marina/anacon

391 rows with ungrouped claimed type.
Remaining rows in valid dataframe: 15959
691 rows in redshift band: 0.2 - 0.30000000000000004
             precision    recall  f1-score   support

          1       0.25      1.00      0.40         1
          2       0.81      0.76      0.79       176
          3       0.30      0.38      0.34        45
          5       0.00      0.00      0.00         0
          6       0.00      0.00      0.00         2
          7       0.00      0.00      0.00         4
         13       0.00      0.00      0.00         1

avg / total       0.68      0.66      0.67       229

0.3


...

/Users/marina/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
/Users/marina/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/marina/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/marina/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/marina/anacon

152 rows in redshift band: 0.5 - 0.6
             precision    recall  f1-score   support

          1       0.50      0.67      0.57         3
          2       0.83      0.83      0.83        41
          3       0.17      0.20      0.18         5
          5       0.00      0.00      0.00         1
          6       0.00      0.00      0.00         1

avg / total       0.71      0.73      0.72        51

0.6
391 rows with ungrouped claimed type.
Remaining rows in valid dataframe: 15959


/Users/marina/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
/Users/marina/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/marina/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/marina/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/marina/anacon

70 rows in redshift band: 0.6 - 0.7
             precision    recall  f1-score   support

          1       1.00      0.25      0.40         4
          2       0.74      0.88      0.80        16
          3       0.25      0.33      0.29         3
          7       0.00      0.00      0.00         1

avg / total       0.69      0.67      0.64        24

0.7
391 rows with ungrouped claimed type.
Remaining rows in valid dataframe: 15959
53 rows in redshift band: 0.7 - 0.7999999999999999
             precision    recall  f1-score   support

          1       0.33      0.67      0.44         3
          2       0.83      0.77      0.80        13
          3       0.00      0.00      0.00         1
          7       0.00      0.00      0.00         1

avg / total       0.66      0.67      0.65        18



/Users/marina/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
/Users/marina/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/marina/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/marina/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/marina/anacon

### Data Profiling 

 Feature Distribution by Claimed Type

In [67]:
# Get distribution of features across claimed type group
type_counts = pd.crosstab(df_rs_band['claimedtype_group'], 
                                    columns = [df_rs_band['masses'],
                                              df_rs_band['velocity'],
                                              df_rs_band['SDSS_SpecMatched'],
                                              df_rs_band['SDSS_FiberID'],
                                              df_rs_band['SDSS_MJD'],
                                              df_rs_band['SDSS_Plate'],
                                              df_rs_band['MPAJHU_BPTClass'],
                                              df_rs_band['WiscPCA_src'],
                                              df_rs_band['HyperLEDA_type'],
                                              df_rs_band['HyperLEDA_bar'],
                                              df_rs_band['HyperLEDA_ring'],
                                               df_rs_band['HyperLEDA_multiple'],
                                               df_rs_band['HyperLEDA_compactness'],
                                               df_rs_band['HyperLEDA_agnclass']
                                                ])
type_counts.to_csv("feature_counts.csv")

Table of the number of claimed types within each range of redshifts.

In [7]:
# Group redshifts into bins of .5
bin_size = 0.1
range_bins = list(np.arange(int(rsmin)-1, int(rsmax) + 2, bin_size))

# Create aggregate frame based on redshift and claimedtype_group
df_agg = valid_df.copy()
df_agg = df_agg[['claimedtype_group','redshift']]
# Split counts into bins of redshift, of bin_size
df_agg['rs_range'] = pd.cut(df_agg['redshift'], bins = range_bins)

# Flip axis so each bin is a column
grouping = df_agg.groupby(['claimedtype_group',
                           'rs_range']).size().reset_index(name = 'counts')
g = pd.DataFrame(grouping)
col_headers = [df_agg.rs_range.unique()]
ct_range = g.pivot(index = 'claimedtype_group', columns = 'rs_range')['counts']

# Save claimedtype ranges to CSV
ct_range.to_csv(output_dir + "ct_range_bin_" + str(bin_size) + ".csv")